In [1]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

3778

In [3]:
with open(files[1], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\0001be30-1ef6-40d8-8f92-13ffbd093cf1.json'

In [5]:
json_data['data']['attributes']

{'id': '00107855-d79b-4efb-808e-68c3a6816077'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Competitive',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 1433161,
 'dateStarted': '2022-08-07T22:08:58.634+00:00',
 'rounds': 15,
 'isRanked': True,
 'queueId': 'competitive',
 'map': 'e2ad5c54-4114-a870-9641-8ea21279579a',
 'mapName': 'Icebox',
 'mapImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/maps/icebox.jpg'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T11:04:22.5577585+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## player_rounds_kills

In [11]:
player_rounds_kills[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [12]:



player_rounds_kills[0]['attributes']

{'round': 1,
 'platformSlug': 'riot',
 'platformUserIdentifier': 'voidedrose#4822',
 'opponentPlatformSlug': 'riot',
 'opponentPlatformUserIdentifier': 'Artx#Art'}

In [13]:
player_rounds_kills[0]['metadata']

{'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'voidedrose#4822',
  'platformUserIdentifier': 'voidedrose#4822',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentPlatformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Artx#Art',
  'platformUserIdentifier': 'Artx#Art',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentLocation': {'x': 1190, 'y': 47},
 'playerLocations': [{'puuid': 'iZWFihGHTi07u3bNglWY1BMEf-8LsPiEalgO1CBR4bssx8li3Enge4Ijam0t8JjxnzGZ5OeLb0R22Q',
   'viewRadians': 4.0423274,
   'location': {'x': 2448, 'y': 1632}},
  {'puuid': '_wKJAZNBbCjKQxuM0gEkItTZryyge6CtWd3Dk4QcrLdFQgIEd4vPmeiJ3pQAkhk-cg2yxQPFeSHO0Q',
   'viewRadians': 4.5714545,
   'location': {'x': 1882, 'y': 3400}},
  {'puuid': '-GLzTJ-D9Z2isNlMjDecZCay37pIpPU90gkEoubOU_NJLsCwKPzkgrWyP5ZXByBgjq4UyvogHTyxbQ',
   'viewRadians': 0.742298,
   'location': {'x': 827, 'y': -802}},
  {'puuid': 'jUQnnrEHxnpeu4DyHd6R

In [14]:
player_rounds_kills[0]['stats']

{'damage': {'rank': None,
  'percentile': None,
  'displayName': None,
  'displayCategory': None,
  'category': None,
  'metadata': {},
  'value': 115,
  'displayValue': '115',
  'displayType': 'Number'}}

In [15]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
print(len(files))
kill_records = list()

for file in files:
    
    
    with open(file, 'r') as f:
        json_data = json.load(f)
    json_data = json.loads(json_data)
    
    if 'data' not in json_data or json_data['data']['metadata']['modeName'] != 'Competitive':
        continue
    
    player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
    for kill in player_rounds_kills:        
        kill_records.append({'weaponName':kill['metadata']['weaponName'],
                            'round':kill['attributes']['round'],
                            'roundTime':kill['metadata']['roundTime']})
kill_df = pd.DataFrame.from_dict(kill_records)
kill_df.head()

3778


,weaponName,round,roundTime
0,Ghost,1,27856
1,Ghost,1,31251
2,Ghost,1,74233
3,None,1,33177
4,Classic,1,15984


In [16]:
kill_df['weaponName'].value_counts().index.tolist()

['Vandal',
 'Phantom',
 'Spectre',
 'Ghost',
 'Classic',
 'Operator',
 'Sheriff',
 'Guardian',
 'Marshal',
 'Odin',
 'Bulldog',
 'Judge',
 'Stinger',
 'Frenzy',
 'Ares',
 'Shorty',
 'Bucky']

In [17]:
kill_df[kill_df['round'] == 1]['weaponName'].value_counts()

Ghost      10802
Classic     8900
Sheriff     2196
Frenzy      1264
Shorty       321
Name: weaponName, dtype: int64

## round_summary

In [18]:
# round_summary[19]

## player_rounds

In [19]:
player_rounds[17]

{'type': 'player-round',
 'attributes': {'round': 2,
  'platformSlug': 'riot',
  'platformUserIdentifier': 'Most Blackie#GANG'},
 'metadata': {'teamId': 'Red',
  'agentKey': 'dade69b4-4f5a-8528-247b-219e5a1facd6',
  'platformInfo': {'platformSlug': 'riot',
   'platformUserId': None,
   'platformUserHandle': 'Most Blackie#GANG',
   'platformUserIdentifier': 'Most Blackie#GANG',
   'avatarUrl': None,
   'additionalParameters': None},
  'hasWon': False},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'score': {'rank': None,
   'percentile': None,
   'displayName': 'Score',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 135,
   'displayValue': '135',
   'displayType': 'Number'},
  'kills': {'rank': None,
   'percentile': None,
   'displayName': 'Kills',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 0,
   'displayValue': '0',
   'displayType': 'Number'},
  'deaths': {'rank': None,
   'percentile': None,
   'displayNa

## round_summary

In [20]:
len(round_summary)

15

In [21]:
# round_summary[22]


In [22]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Blue'

## player_summary

In [23]:
len(player_summary)

10

In [24]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [25]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'voidedrose#4822'}

In [26]:
player_summary[0]['metadata']

{'partyId': 'e853526d-a4ff-4e57-9a7f-23e9a8de3030',
 'teamId': 'Blue',
 'agentKey': 'a3bfb853-43b2-7238-a4f1-ad90e9e46bcc',
 'agentName': 'Reyna',
 'agentColor': '#f650d3',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/a3bfb853-43b2-7238-a4f1-ad90e9e46bcc/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/reyna_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'voidedrose#4822',
  'platformUserIdentifier': 'voidedrose#4822',
  'avatarUrl': None,
  'additionalParameters': None}}

In [27]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [28]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/6.png'},
 'value': 'Bronze 1',
 'displayValue': 'Bronze 1',
 'displayType': 'String'}

In [29]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 5262,
 'displayValue': '5,262',
 'displayType': 'Number'}

In [30]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 350.8,
 'displayValue': '351',
 'displayType': 'Number'}

## Win prediction data set creation

In [31]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']
weapons = ['Vandal',
             'Phantom',
             'Spectre',
             'Ghost',
             'Classic',
             'Operator',
             'Sheriff',
             'Guardian',
             'Marshal',
             'Odin',
             'Judge',
             'Bulldog',
             'Stinger',
             'Frenzy',
             'Ares',
             'Shorty',
             'Bucky']
ranks = [
    'Unranked',
    'Iron 1',
    'Iron 2',
    'Iron 3',
    'Bronze 1',
    'Bronze 2',
    'Bronze 3',
    'Silver 1',
    'Silver 2',
    'Silver 3',
    'Gold 1',
    'Gold 2',
    'Gold 3',
    'Platinum 1',
    'Platinum 2',
    'Platinum 3',
    'Diamond 1',
    'Diamond 2',
    'Diamond 3',
    'Ascendant 1',
    'Ascendant 2',
    'Ascendant 3',
    'Immortal 1',
    'Immortal 2',
    'Immortal 3',
    'Radiant',
        ]

In [32]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for j in roles_list:
            if j == agent_role:
                new_record[f'role_{j}'] = 1
            else:
                new_record[f'role_{j}'] = 0
        
        for j in all_agents_list:
            if j == agent:
                new_record[f'agent_{j}'] = 1
            else:
                new_record[f'agent_{j}'] = 0
        
        weapon_kills_dict = {j: 0 for j in weapons}
        for j in player_rounds_kills:
            if 'platformInfo' in j and i['attributes']['platformUserIdentifier'] != j['platformInfo']['platformUserHandle']:
                if j['metadata']['weaponName'] in weapons:
                    weapon_kills_dict[j['metadata']['weaponName']] += 1
            
        data.append(new_record)
            
    return data
        
        
# processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills)
# processed_match_records[8]

In [33]:
player_summary[0]

{'type': 'player-summary',
 'attributes': {'platformSlug': 'riot',
  'platformUserIdentifier': 'voidedrose#4822'},
 'metadata': {'partyId': 'e853526d-a4ff-4e57-9a7f-23e9a8de3030',
  'teamId': 'Blue',
  'agentKey': 'a3bfb853-43b2-7238-a4f1-ad90e9e46bcc',
  'agentName': 'Reyna',
  'agentColor': '#f650d3',
  'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/a3bfb853-43b2-7238-a4f1-ad90e9e46bcc/displayicon.png',
  'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/reyna_portrait.png',
  'countryCode': None,
  'platformInfo': {'platformSlug': 'riot',
   'platformUserId': None,
   'platformUserHandle': 'voidedrose#4822',
   'platformUserIdentifier': 'voidedrose#4822',
   'avatarUrl': None,
   'additionalParameters': None}},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'rank': {'rank': None,
   'percentile': None,
   'displayName': 'Rating',
   'displayCategory': None,
   'category': 'mmr',
   'metadata': {'iconUrl': 'https://trackercdn.c

In [34]:


def create_agent_row(agent_list, map_pick, game_win, rank_list):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
    
    rank_avg = sum([ranks.index(i) for i in rank_list])/max(len(rank_list), 1)
    row['rank_avg'] = rank_avg
    
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    rank_list_team_red = []
    rank_list_team_blue = []
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
            rank_list_team_red.append(i['stats']['rank']['value'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            rank_list_team_blue.append(i['stats']['rank']['value'])

    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team), rank_list_team_red)
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team), rank_list_team_blue)
    
    return [row1, row2]
    
    




In [35]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
            
        if json_data['data']['metadata']['modeName'] != 'Competitive':
            continue
        
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\e69ee2e2-e082-441b-8f24-000b12441f84.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\f7145f0c-05c0-43ca-b69e-b191a902d9bc.json


(35050, 7010)

In [36]:
ranks.index('Unranked')

0

In [37]:
ranks

['Unranked',
 'Iron 1',
 'Iron 2',
 'Iron 3',
 'Bronze 1',
 'Bronze 2',
 'Bronze 3',
 'Silver 1',
 'Silver 2',
 'Silver 3',
 'Gold 1',
 'Gold 2',
 'Gold 3',
 'Platinum 1',
 'Platinum 2',
 'Platinum 3',
 'Diamond 1',
 'Diamond 2',
 'Diamond 3',
 'Ascendant 1',
 'Ascendant 2',
 'Ascendant 3',
 'Immortal 1',
 'Immortal 2',
 'Immortal 3',
 'Radiant']

In [38]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'strawberry milk#우유갠디', 'count': 38},
 {'user': 'wakefull#icy', 'count': 35},
 {'user': 'CV WorstNub#2822', 'count': 33},
 {'user': '32Equr#6990', 'count': 30},
 {'user': 'Thug Twerker#money', 'count': 30},
 {'user': 'POPPIN#Uwu', 'count': 30},
 {'user': 'Cowpico#007', 'count': 28},
 {'user': 'tenz jr#333', 'count': 28},
 {'user': 'Tomioka#0818', 'count': 27},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'Daddy Brim#OTP', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'CandyMan#7591', 'count': 24},
 {'user': 'Tan#0008', 'count': 24},
 {'user': 'Knights ban#xuf', 'count': 24},
 {'user': 'ZenitsuAG#7485', 'count': 24},
 {'user': 'I bark for mommy#6969', 'count': 24}]

In [39]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

6866

In [40]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
1,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
2,Boldzero#101,2022-07-27T05:21:43.67+00:00,1,Red,Ascent,Bronze 3,1052,210.400000,0.800000,4,...,0,1,0,0,0,0,0,0,0,0
3,Migegg#8765,2022-07-29T23:40:51.769+00:00,1,Red,Pearl,Bronze 2,4763,250.684211,0.842105,16,...,0,0,0,0,0,0,0,0,0,0
4,DarkPegasis123#NA1,2022-08-06T17:49:51.751+00:00,0,Blue,Pearl,Gold 3,2898,144.900000,0.400000,8,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6861,frost2094#4441,2022-08-06T20:27:11.453+00:00,1,Red,Icebox,Bronze 3,472,78.666667,0.333333,2,...,0,0,0,0,0,0,0,0,0,0
6862,SmokedSalmon#4492,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Gold 2,4892,188.153846,0.615385,16,...,0,0,0,1,0,0,0,0,0,0
6863,Mrpancake#31798,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Silver 2,4089,157.269231,0.538462,14,...,0,0,0,0,0,0,0,0,0,0
6864,August#SUI,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Gold 1,7741,297.730769,1.038462,27,...,0,1,0,0,0,0,0,0,0,0


In [41]:
all_records_df['name'].nunique()

501

In [42]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [43]:
all_records_df['map_pick'].value_counts(normalize=True)

Bind        0.147539
Ascent      0.147247
Haven       0.145354
Fracture    0.145063
Breeze      0.143315
Pearl       0.134576
Icebox      0.131955
Split       0.004952
Name: map_pick, dtype: float64

In [44]:
all_records_df['killsPerRound'].describe()

count    6866.000000
mean        0.725973
std         0.307837
min         0.000000
25%         0.500000
50%         0.695652
75%         0.904762
max         2.500000
Name: killsPerRound, dtype: float64

In [45]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 2       0.110545
Bronze 1       0.105156
Bronze 3       0.095835
Silver 1       0.092776
Silver 3       0.082872
Silver 2       0.068890
Radiant        0.063938
Iron 3         0.062627
Gold 1         0.056656
Gold 2         0.051704
Gold 3         0.047189
Unranked       0.036557
Iron 2         0.031314
Platinum 1     0.029420
Platinum 2     0.019516
Platinum 3     0.015293
Immortal 3     0.012525
Iron 1         0.004078
Diamond 3      0.003204
Diamond 1      0.003059
Diamond 2      0.002913
Ascendant 1    0.001311
Immortal 2     0.001165
Immortal 1     0.000874
Ascendant 2    0.000437
Ascendant 3    0.000146
Name: rank, dtype: float64

In [46]:
# all_records_df[all_records_df['rank'].str.contains('Diamond')]['name'].value_counts().index.tolist()

In [47]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [48]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

len(training_players), len(val_players)

(375, 126)

In [49]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [50]:
# training_data_df.isna().sum()

In [51]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5335720680393913

In [52]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.8,5505.0,256.708105,0.869198,18.6,15.8,6.2,1.237582,3569.8,165.801134,5.8,3.8,1.2,0.4,0.0,1.6,13.0,17.8,16.8,3.8,0.604211,0.826548,0.777891,0.176058,2.4,2.4,3.0,2.0,66.909091,0.135841,0.093147,62.0,21.638809,72.705190,0.0,0.2,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.8,5164.4,247.781438,0.839198,17.4,13.8,5.4,1.318254,3409.6,163.484467,6.4,3.0,1.2,0.4,0.0,1.6,10.8,19.8,15.4,3.8,0.510877,0.983214,0.734558,0.182725,2.6,2.6,2.8,1.6,61.909091,0.125841,0.076480,64.0,18.941839,72.371856,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.8,4761.2,229.454165,0.775561,16.0,13.6,4.8,1.252464,3142.4,151.339012,5.4,2.8,1.2,0.4,0.0,1.6,8.0,24.0,14.4,3.8,0.383604,1.174123,0.689103,0.182725,3.0,3.0,1.6,1.2,51.000000,0.071296,0.058298,58.2,16.463207,66.917311,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.8,5021.0,282.412819,0.983254,17.4,11.6,4.0,1.774687,3269.8,184.162089,4.2,2.6,1.6,0.6,0.2,2.4,9.2,19.0,14.2,3.0,0.521104,1.010662,0.782372,0.161571,1.8,1.8,1.8,1.6,47.500000,0.103026,0.088106,88.0,17.535730,69.032695,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.8,4747.8,263.388258,0.915584,16.4,12.2,3.8,1.641353,3206.4,177.535022,5.6,2.4,1.2,0.4,0.2,1.8,10.2,15.2,13.0,2.6,0.560703,0.805649,0.712197,0.137511,1.0,1.0,2.4,2.2,38.928571,0.130595,0.116677,86.4,21.438169,68.681818,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,0.4,4555.4,215.127945,0.735683,15.6,15.2,4.4,1.049658,2948.6,138.570038,5.6,3.2,0.6,0.2,0.2,1.0,10.2,13.0,14.2,2.6,0.530075,0.619242,0.645746,0.119286,0.6,0.6,2.4,2.4,43.238095,0.122055,0.110219,59.0,31.576365,64.827068,0.2,0.2,0.2,0.4,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0
11,0.4,4811.4,211.477945,0.714850,16.4,16.8,5.4,0.978547,3194.6,140.245038,6.6,3.4,0.4,0.2,0.2,0.8,8.6,12.2,19.8,2.8,0.396742,0.565075,0.849912,0.119286,1.2,1.2,2.2,2.6,39.238095,0.097055,0.114386,55.0,34.558581,

In [53]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
28,past_5_games_avg_esr,0.031883
21,past_5_games_avg_ability1CastsPerRound,0.031966
29,past_5_games_avg_firstKillsPerRound,0.032181
30,past_5_games_avg_firstDeathsPerRound,0.032283
31,past_5_games_avg_econRating,0.033208
20,past_5_games_avg_grenadeCastsPerRound,0.033250
22,past_5_games_avg_ability2CastsPerRound,0.033592
23,past_5_games_avg_ultimateCastsPerRound,0.033716
32,past_5_games_avg_hsAccuracy,0.035168
33,past_5_games_avg_kast,0.036198


In [54]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5237242614145031

In [55]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

columns_list = training_data_df.columns.tolist()

for i in columns_list:
    df_interaction[i] =  all_data_df[i]
    for j in columns_list:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_score_mul_past_5_games_avg_singleKills,past_5_games_avg_score_mul_past_5_games_avg_doubleKills,past_5_games_avg_score_mul_past_5_games_avg_tripleKills,past_5_ga

In [56]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(20)

,column,slope,intercept,rvalue,r2,pvalue,stderr
352,past_5_games_avg_assists_mul_past_5_games_avg_econRating,19.317072,275.946752,0.083977,0.007052,2.794212e-08,3.471780
328,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.438942,5.211033,0.081423,0.006630,7.267336e-08,0.081380
332,past_5_games_avg_assists_mul_past_5_games_avg_doubleKills,1.033470,12.442771,0.078763,0.006204,1.909200e-07,0.198120
330,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,42.348446,655.345911,0.078277,0.006127,2.270201e-07,8.169081
117,past_5_games_avg_scorePerRound_mul_past_5_games_avg_assists,64.423900,999.215747,0.077270,0.005971,3.239551e-07,12.590413
171,past_5_games_avg_killsPerRound_mul_past_5_games_avg_assists,0.216242,3.421804,0.075991,0.005775,5.057163e-07,0.042976
329,past_5_games_avg_assists_mul_past_5_games_avg_damage,883.733801,13539.779431,0.072808,0.005301,1.486066e-06,183.355656
62,past_5_games_avg_score_mul_past_5_games_avg_assists,1355.003530,20618.300037,0.072659,0.005279,1.561096e-06,281.711926
224,past_5_games_avg_kills_mul_past_5_games_avg_assists,4.529595,70.512606,0.071712,0.005143,2.132273e-06,0.954231
382,past_5_games_avg_kdRatio_mul_past_5_games_avg_doubleKills,0.311594,3.041415,0.070949,0.005034,2.733526e-06,0.066352


In [57]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
7,past_5_games_avg_kdRatio,0.058577,1.093401,0.062827,3.947259e-03,0.000033,0.014094
11,past_5_games_avg_doubleKills,0.123809,2.572385,0.062744,3.936791e-03,0.000034,0.029828
8,past_5_games_avg_damage,91.128619,2779.881835,0.059442,3.533302e-03,0.000086,23.179408
1,past_5_games_avg_score,140.364568,4235.877890,0.058113,3.377078e-03,0.000123,36.522370
9,past_5_games_avg_damagePerRound,3.925129,136.322610,0.057901,3.352570e-03,0.000130,1.025044
6,past_5_games_avg_assists,0.193391,4.813578,0.057515,3.307957e-03,0.000145,0.050845
2,past_5_games_avg_scorePerRound,6.006454,207.945813,0.055795,3.113116e-03,0.000227,1.627984
4,past_5_games_avg_kills,0.469248,14.570734,0.053761,2.890205e-03,0.000383,0.132013
31,past_5_games_avg_econRating,1.708144,58.136972,0.052168,2.721486e-03,0.000568,0.495264
3,past_5_games_avg_killsPerRound,0.020321,0.716141,0.050762,2.576734e-03,0.000798,0.006056



## Agent pick prediction data set creation

In [58]:
all_records, all_agent_records = get_all_processed_data()


error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\e69ee2e2-e082-441b-8f24-000b12441f84.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\f7145f0c-05c0-43ca-b69e-b191a902d9bc.json


In [59]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [60]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [61]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,rank_avg,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel,game_win
0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,4.4,0,1,2,2,0,0,0,2,2,4,0
1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,3.6,0,0,2,3,0,0,0,0,0,6,1
2,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,4.6,1,0,2,2,0,2,2,0,0,4,1
3,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0.0,0,1,3,1,0,0,0,3,1,3,0
4,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,6.6,1,1,2,1,1,2,1,2,1,2,1


In [62]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,rank_avg,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel
722,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,5.6,1,1,2,1,1,2,1,2,1,2
680,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,7.6,1,1,3,0,1,3,0,3,0,0
986,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,24.6,1,1,2,1,1,2,1,2,1,2
6591,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,0,0,0,0,0,0,5.2,1,1,2,1,1,2,1,2,1,2
517,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,24.4,1,1,2,1,1,2,1,2,1,2


In [63]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

0.4985787379192723

In [64]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
17,agent_Viper,0.024886
18,agent_Yoru,0.025413
14,agent_Sage,0.025718
10,agent_Omen,0.026175
3,agent_Chamber,0.026786
11,agent_Phoenix,0.027343
13,agent_Reyna,0.030593
12,agent_Raze,0.032424
6,agent_Jett,0.034404
27,rank_avg,0.274614


In [65]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
5,agent_Fade,0.037072,0.195398,0.045207,2.043675e-03,0.001023,0.011281
9,agent_Neon,-0.027206,0.207846,-0.034379,1.181913e-03,0.012523,0.010891
13,agent_Reyna,0.026891,0.576386,0.027335,7.471980e-04,0.047121,0.013543
32,role_Controller*role_Initiator,-0.038005,0.824972,-0.026382,6.960267e-04,0.055364,0.019831
16,agent_Sova,-0.018487,0.203320,-0.023370,5.461653e-04,0.089661,0.010891
37,role_Duelist*role_Sentinel,0.057359,2.204074,0.022911,5.249044e-04,0.096149,0.034468
28,role_Controller,-0.019147,0.917012,-0.020621,4.252354e-04,0.134261,0.012784
7,agent_KAY/O,-0.015287,0.218031,-0.018752,3.516266e-04,0.173287,0.011225
15,agent_Skye,-0.015758,0.246322,-0.018488,3.418218e-04,0.179403,0.011735
0,agent_Astra,-0.007125,0.048284,-0.017233,2.969633e-04,0.210793,0.005693


In [66]:
def create_interactions(df: pd.DataFrame) -> pd.DataFrame:
    df_interaction = pd.DataFrame(index = df.index)

    df_interaction['game_win'] = df['game_win']

    columns_list = sorted(df.columns.tolist())

    for i in columns_list:
        df_interaction[i] = df[i]
        for j in columns_list:
            if  i == 'game_win' or j == 'game_win':
                continue
            if columns_list.index(i) >=columns_list.index(j):
                continue
            
            if 'rank' in i:
                df_interaction[f'{i}_mul_{j}'] = df[i]*df[j]
            elif 'role' in i:
                df_interaction[f'{i}_mul_{j}'] = df[i]*df[j]
                df_interaction[f'{i}_max_{j}'] = df[[i,j]].max(axis = 1)
            else:
                df_interaction[f'{i}_max_{j}'] = df[i]*df[j]
                
    return df_interaction

In [67]:
train_df, val_df = train_test_split(all_agent_records_df)


In [68]:
fi_list = list()

df_interaction_train = create_interactions(train_df)

for i in df_interaction_train.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_interaction_train['game_win'], df_interaction_train[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

,column,slope,intercept,rvalue,r2,pvalue,stderr
152,agent_Cypher_max_agent_Omen,-0.013196,0.034312,-0.040219,0.001618,0.003483,0.004515
173,agent_Cypher_max_role_Controller*role_Sentinel,-0.038102,0.139535,-0.039540,0.001563,0.004077,0.013260
59,agent_Breach_max_map_Fracture,0.012595,0.020587,0.038911,0.001514,0.004707,0.004454
187,agent_Fade_max_agent_Raze,0.017956,0.047655,0.038825,0.001507,0.004799,0.006364
253,agent_KAY/O_max_agent_Skye,-0.012827,0.035074,-0.038461,0.001479,0.005210,0.004589
764,role_Duelist*role_Sentinel_mul_role_Initiator*role_Duelist,0.190720,2.627907,0.038072,0.001449,0.005684,0.068936
762,role_Duelist*role_Sentinel_mul_role_Initiator,0.090492,1.517728,0.037504,0.001407,0.006446,0.033205
780,role_Initiator*role_Duelist_mul_role_Sentinel,0.090492,1.517728,0.037504,0.001407,0.006446,0.033205
757,role_Duelist_mul_role_Initiator*role_Sentinel,0.090492,1.517728,0.037504,0.001407,0.006446,0.033205
766,role_Duelist*role_Sentinel_mul_role_Initiator*role_Sentinel,0.144105,1.994281,0.035128,0.001234,0.010727,0.056459


In [69]:
df_interaction_train.shape

(5275, 787)

In [70]:
results = list()


In [ ]:
import random

counter = 0

while True:
    try:
        counter += 1
        if counter > 2000:
            break
        i = random.randint(10, 100)
        j = random.randint(10, 50) * .01
        max_depth = random.randint(2, 8)
        
        train_df, val_df = train_test_split(all_agent_records_df)

        model = RandomForestClassifier(n_estimators=i, max_features = j, max_depth=max_depth)

        df_train_int = create_interactions(train_df)
        df_val_int = create_interactions(val_df)


        model.fit(df_train_int.drop('game_win', axis = 1), train_df['game_win'])

        preds = model.predict(df_val_int.drop('game_win', axis = 1))
        score= accuracy_score(preds, val_df['game_win'])

        results.append({'n_estimators':i,
                       'max_features':j,
                        'max_depth':max_depth,
                       'score':score})
        if score >= sorted(results, key = lambda x: x['score'], reverse = True)[0]['score'] or counter%40 == 0:
            print(counter, i, j, score, sorted(results, key = lambda x: x['score'], reverse = True)[:3])
    except:
        break
        
            
sorted(results, key = lambda x: x['score'], reverse = True)[:10]

1 37 0.31 hits#glhf 0.4923251847640705 [{'n_estimators': 37, 'max_features': 0.31, 'max_depth': 7, 'score': 0.4923251847640705}]
2 48 0.16 hits#glhf 0.4985787379192723 [{'n_estimators': 48, 'max_features': 0.16, 'max_depth': 7, 'score': 0.4985787379192723}, {'n_estimators': 37, 'max_features': 0.31, 'max_depth': 7, 'score': 0.4923251847640705}]
3 42 0.17 hits#glhf 0.5002842524161455 [{'n_estimators': 42, 'max_features': 0.17, 'max_depth': 8, 'score': 0.5002842524161455}, {'n_estimators': 48, 'max_features': 0.16, 'max_depth': 7, 'score': 0.4985787379192723}, {'n_estimators': 37, 'max_features': 0.31, 'max_depth': 7, 'score': 0.4923251847640705}]
4 86 0.18 hits#glhf 0.5054007959067652 [{'n_estimators': 86, 'max_features': 0.18, 'max_depth': 7, 'score': 0.5054007959067652}, {'n_estimators': 42, 'max_features': 0.17, 'max_depth': 8, 'score': 0.5002842524161455}, {'n_estimators': 48, 'max_features': 0.16, 'max_depth': 7, 'score': 0.4985787379192723}]
5 40 0.23 hits#glhf 0.526435474701535 [

In [ ]:
# sorted(results, key = lambda x: x['score'], reverse = True)

In [ ]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df, random_state = 1)

n_estimators = 40
max_features = 0.33
max_depth = 10

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)

df_train_int = create_interactions(train_df)
df_val_int = create_interactions(val_df)

model.fit(df_train_int.drop('game_win', axis = 1), train_df['game_win'])

preds = model.predict(df_val_int.drop('game_win', axis = 1))
accuracy_score(preds, val_df['game_win'])


In [ ]:
fi_list = list()
for i, j in zip(df_train_int.columns, model.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

In [ ]:
fi_list = list()


for i in df_train_int.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_train_int['game_win'], df_train_int[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

In [ ]:
fi_list = list()

for i in df_val_int.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_val_int['game_win'], df_val_int[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

In [ ]:
# df_train_int.corr()[[i for i in df_train_int.columns if 'pca' not in i]]

In [ ]:

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)

df_all = create_interactions(all_agent_records_df)

model.fit(df_all.drop('game_win', axis = 1), all_agent_records_df['game_win'])

preds = model.predict(df_all.drop('game_win', axis = 1))
preds.shape, df_all.shape

In [ ]:
def predict_best_lineup(model, map_pick, current_agent_list, rank_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None, rank_list))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df_interactions = create_interactions(features_df)

    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index

    inputs_df['win_prob'] = model.predict_proba(features_df_interactions.drop('game_win', axis = 1))[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = []
rank_list= ['Bronze 2']

predict_best_lineup(model, map_pick, current_agent_list, rank_list).head()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []
rank_list= ['Gold 2']

predict_best_lineup(model, map_pick, current_agent_list, rank_list).head()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []
rank_list= ['Radiant']

predict_best_lineup(model, map_pick, current_agent_list, rank_list).head()

In [ ]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [ ]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()